In [78]:
import numpy as np
from scipy.stats import gmean

In [79]:
def check_independence_assumption(ground_truth : np.ndarray, single_site_frequencies: np.array, sequence_effects : np.ndarray, frequencies : np.ndarray) -> tuple:
    
    # get number_states and sequence length
    number_states, sequence_length = ground_truth.shape

    # compute the geometric mean of the sequence effects
    gmean_sequence_effects = gmean(sequence_effects, weights=frequencies)

    # compute the products of the geometric means of the single site effects (columns are positions, rows are states)
    gmean_single_site_effects = np.prod(gmean(ground_truth, axis=1, weights=single_site_frequencies), axis=0)

    # check if the geometric mean of the sequence effects is equal to the product of the geometric means of the single site effects
    print("gmean sequence effects: ", np.round(gmean_sequence_effects, 5))
    print("product of gmean single site effects: ", np.round(gmean_single_site_effects, 5))
    print("gmean sequence effects is equal to product of gmean single site effects: ", np.isclose(gmean_sequence_effects, gmean_single_site_effects))

    return gmean_sequence_effects, gmean_single_site_effects

In [80]:
number_states = 4
sequence_length = 5
p_effect = 0.1
p_state_change = 1/4

In [81]:
def generate_ground_truth(number_states : int, sequence_length : int, p_effect : float) -> np.ndarray:
    # default state has value 1
    default_state = np.ones(sequence_length)
    # mutant states are drawn from a log-normal distribution
    mutant_states = np.round(np.random.lognormal(mean=0, sigma=1, size=(number_states-1, sequence_length)),2)
    # set mutant states to 1 with probability 1 - p_effect
    mutant_states = np.where(np.random.rand(*mutant_states.shape) < 1-p_effect, 1, mutant_states)

    return np.row_stack((default_state, mutant_states))

In [82]:
def generate_sequences(ground_truth : np.ndarray, p_state_change : float, random : bool = False, pruning : bool = False) -> np.ndarray:

    # get number_states and sequence length
    number_states, sequence_length = ground_truth.shape

    # create every possible sequence
    sequences = np.array(np.meshgrid(*[np.arange(number_states)]*sequence_length)).T.reshape(-1, sequence_length)

    if not random: 
        # calculate the probability of each sequence as p_state_change**number of state changes * (1-p_state_change)**(sequence_length - number of state changes)
        state_changes = np.sum(sequences != np.zeros(sequence_length), axis=1)
        frequencies = (p_state_change/(number_states-1))**state_changes * (1-p_state_change)**(sequence_length - state_changes)

    if random:
        # generate a random frequency for each sequence
        frequencies = np.random.rand(sequences.shape[0])
        # normalize frequencies
        frequencies = frequencies / np.sum(frequencies)

    if pruning:
        # set frequency < 0.00001 to 0
        frequencies = np.where(frequencies < 0.001, 0, frequencies)
        #normalize frequencies
        frequencies = frequencies / np.sum(frequencies)


    # compute effect of each unique sequence
    # effect of a sequence is the product of the effects of the states per position
    sequence_effects = np.array([np.prod([ground_truth[int(sequences[i,j]), j] for j in range(sequence_length)]) for i in range(sequences.shape[0])])

    # print("sequences: ", sequences)
    print("frequencies: ", frequencies)
    print("sequence_effects: ", sequence_effects)

    return sequences, frequencies, sequence_effects

frequencies:  [0.00025228 0.00125629 0.00053628 ... 0.00159991 0.00154486 0.00096974]
sequence_effects:  [1.   1.   2.26 ... 1.   2.26 1.  ]


In [83]:
def compute_single_site_frequencies(sequences : np.ndarray, frequencies : np.ndarray, number_states : int, sequence_length : int) -> np.ndarray:
    # create a matrix of frequencies of each state at each position
    single_site_frequencies = np.zeros((number_states, sequence_length))
    for i in range(number_states):
        for j in range(sequence_length):
            # get the row indices of the unique sequences that have state i at position j
            row_indices = np.where(sequences[:,j] == i)[0]
            # sum the counts of these sequences
            single_site_frequencies[i,j] = np.sum(frequencies[row_indices])

    return single_site_frequencies

single_site_frequencies:  [[0.25449625 0.25575672 0.2457438  0.24942524 0.25240111]
 [0.23714238 0.25023082 0.25125667 0.24662799 0.24226664]
 [0.24473777 0.25517003 0.251624   0.25257615 0.2443919 ]
 [0.26362361 0.23884243 0.25137553 0.25137062 0.26094035]]


In [84]:
def assumption_test(number_states : int, sequence_length : int, p_effect : float, p_state_change : float, random : bool = False, pruning : bool = False) -> None:
    # generate ground truth
    ground_truth = generate_ground_truth(number_states, sequence_length, p_effect)

    # generate sequences
    sequences, frequencies, sequence_effects = generate_sequences(ground_truth, p_state_change, random, pruning)

    # compute single site frequencies
    single_site_frequencies = compute_single_site_frequencies(sequences, frequencies, number_states, sequence_length)

    # check independence assumption
    gmean_sequence_effects, gmean_single_site_effects = check_independence_assumption(ground_truth, single_site_frequencies, sequence_effects, frequencies)

    return gmean_sequence_effects, gmean_single_site_effects

gmean sequence effects:  1.23128
product of gmean single site effects:  1.18134
gmean sequence effects is equal to product of gmean single site effects:  False
